# Install and init DVC

Prerequisites: 
-  DVC and requirements.txt packages installed (if not - check README.md file for instructions)
-  A project repository is a Git repo 



## Install with pip

In [1]:
!pip install "dvc==1.0.2"

## Checkout branch `tutorial`

In [2]:
!git checkout -b dvc-tutorial

Switched to a new branch 'dvc-tutorial'


## Initialize DVC

References: 
- https://dvc.org/doc/get-started/initialize 

In [3]:
!dvc init


You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|              https://dvc.org/doc/user-guide/analytics               |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: https://dvc.org/doc
- Get help and share ideas: https://dvc.org/chat
- Star us on GitHub: https://github.com/iterative/dvc


## Commit changes

In [4]:
%%bash

git add .
git commit -m "Initialize DVC"

[dvc-tutorial f285905] Initialize DVC
 6 files changed, 128 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/default.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/smooth.json


# Build automated pipelines

## Create `data_load` stage


In [94]:
!mkdir -p data

In [95]:
!dvc run -n data_load \
    -d src/data_load.py \
    -o data/iris.csv \
    -o data/classes.json \
    -p data_load \
    python src/data_load.py \
        --config=params.yaml

Restored stage 'data_load' from run-cache                                       
Skipping run, checking out outputs
Creating 'dvc.yaml'
Adding stage 'data_load' in 'dvc.yaml'
Generating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml .dvc/.gitignore


In [96]:
%%bash

du -sh data/*

4.0K	data/classes.json
4.0K	data/cm.csv
4.0K	data/iris.csv
8.0K	data/iris_featurized.csv
4.0K	data/metrics.json
8.0K	data/model.joblib
4.0K	data/test.csv
8.0K	data/train.csv


In [97]:
!tree -I venv-dvc-3-automate-experiments

.
├── README.md
├── data
│   ├── classes.json
│   ├── cm.csv
│   ├── iris.csv
│   ├── iris_featurized.csv
│   ├── metrics.json
│   ├── model.joblib
│   ├── test.csv
│   └── train.csv
├── dvc-3-automate-experiments.ipynb
├── dvc.lock
├── dvc.yaml
├── params.yaml
├── requirements.txt
└── src
    ├── __init__.py
    ├── data_load.py
    ├── evaluate.py
    ├── featurization.py
    ├── split_dataset.py
    └── train.py

2 directories, 20 files


## dvc.yaml

In [98]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv


## params.yaml

In [99]:
!cat params.yaml


data_load:
  raw_data_path: data/iris.csv
  classes_names_path: data/classes.json

featurize:
  features_path: data/iris_featurized.csv
  target_column: target


data_split:
  test_size: 0.2
  train_path: data/train.csv
  test_path: data/test.csv


train:
  model_path: data/model.joblib


evaluate:
  metrics_file: data/metrics.json
  confusion_matrix: data/cm.csv


## Reproduce a pipeline

In [100]:
!dvc repro

Stage 'data_load' didn't change, skipping                                       
Data and pipelines are up to date.


## Change params.yaml and reproduce 

Add a new line into `data_load` section:
    `dummy_param: dummy_value`

In [101]:
!dvc repro

Stage 'data_load' didn't change, skipping                                       
Data and pipelines are up to date.


# Build end-to-end Machine Learning pipeline
Stages 
- extract features 
- split dataset 
- train 
- evaluate 


## Add feature extraction stage

In [103]:
!dvc run -n feature_extraction \
    -d src/featurization.py \
    -d data/iris.csv \
    -o data/iris_featurized.csv \
    -p data_load,featurize \
    python src/featurization.py \
        --config=params.yaml

Restored stage 'feature_extraction' from run-cache                              
Skipping run, checking out outputs
Adding stage 'feature_extraction' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [104]:
!ls 

README.md                        params.yaml
data                             requirements.txt
dvc-3-automate-experiments.ipynb src
dvc.lock                         venv-dvc-3-automate-experiments
dvc.yaml


In [105]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv
  feature_extraction:
    cmd: python src/featurization.py --config=params.yaml
    deps:
    - data/iris.csv
    - src/featurization.py
    params:
    - data_load
    - featurize
    outs:
    - data/iris_featurized.csv


In [106]:
import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [107]:
!git status -s

?? .dvc/
?? dvc.lock
?? dvc.yaml


In [108]:
%%bash
git add .
git commit -m "Add stage features_extraction"

[dev 0ae7569] Add stage features_extraction
 3 files changed, 56 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml


## Add split train/test stage

In [109]:
!dvc run -n split_dataset \
    -d src/split_dataset.py \
    -d data/iris_featurized.csv \
    -o data/train.csv \
    -o data/test.csv \
    -p featurize,data_split \
        python src/split_dataset.py \
            --config=params.yaml

Restored stage 'split_dataset' from run-cache                                   
Skipping run, checking out outputs
Adding stage 'split_dataset' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [110]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv
  feature_extraction:
    cmd: python src/featurization.py --config=params.yaml
    deps:
    - data/iris.csv
    - src/featurization.py
    params:
    - data_load
    - featurize
    outs:
    - data/iris_featurized.csv
  split_dataset:
    cmd: python src/split_dataset.py --config=params.yaml
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    params:
    - data_split
    - featurize
    outs:
    - data/test.csv
    - data/train.csv


In [111]:
%%bash
git add .
git commit -m "Add stage split_dataset"

[dev e39a9d3] Add stage split_dataset
 2 files changed, 32 insertions(+)


## Add train stage

In [112]:
!dvc run -n train \
    -d src/train.py \
    -d data/train.csv \
    -o data/model.joblib \
    -p data_split,train \
        python src/train.py \
            --config=params.yaml

Restored stage 'train' from run-cache                                           
Skipping run, checking out outputs
Adding stage 'train' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml


In [113]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv
  feature_extraction:
    cmd: python src/featurization.py --config=params.yaml
    deps:
    - data/iris.csv
    - src/featurization.py
    params:
    - data_load
    - featurize
    outs:
    - data/iris_featurized.csv
  split_dataset:
    cmd: python src/split_dataset.py --config=params.yaml
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    params:
    - data_split
    - featurize
    outs:
    - data/test.csv
    - data/train.csv
  train:
    cmd: python src/train.py --config=params.yaml
    deps:
    - data/train.csv
    - src/train.py
    params:
    - data_split
    - train
    outs:
    - data/model.joblib


In [114]:
%%bash
git add .
git commit -m "Add stage train"

[dev d084d1b] Add stage train
 2 files changed, 28 insertions(+)


## Add evaluate stage

In [115]:
!dvc run -n evaluate \
    -d src/evaluate.py \
    -d data/test.csv \
    -d data/model.joblib \
    -d data/classes.json \
    -m data/metrics.json \
    --plots data/cm.csv \
    -p data_load,data_split,train,evaluate \
        python src/evaluate.py \
            --config=params.yaml

Restored stage 'evaluate' from run-cache                                        
Skipping run, checking out outputs
Adding stage 'evaluate' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [116]:
!cat dvc.yaml

stages:
  data_load:
    cmd: python src/data_load.py --config=params.yaml
    deps:
    - src/data_load.py
    params:
    - data_load
    outs:
    - data/classes.json
    - data/iris.csv
  feature_extraction:
    cmd: python src/featurization.py --config=params.yaml
    deps:
    - data/iris.csv
    - src/featurization.py
    params:
    - data_load
    - featurize
    outs:
    - data/iris_featurized.csv
  split_dataset:
    cmd: python src/split_dataset.py --config=params.yaml
    deps:
    - data/iris_featurized.csv
    - src/split_dataset.py
    params:
    - data_split
    - featurize
    outs:
    - data/test.csv
    - data/train.csv
  train:
    cmd: python src/train.py --config=params.yaml
    deps:
    - data/train.csv
    - src/train.py
    params:
    - data_split
    - train
    outs:
    - data/model.joblib
  evaluate:
    cmd: python src/evaluate.py --config=params.yaml
    deps:
    - data/classes.json
    - data/model.joblib

    - src/evaluate.py
    params:
    - d

In [117]:
%%bash
git add .
git commit -m "Add stage evaluate"

[dev ecf5bc5] Add stage evaluate
 2 files changed, 46 insertions(+)


# Experimenting with reproducible pipelines

## How reproduce experiments?

> The most exciting part of DVC is reproducibility.
>> Reproducibility is the time you are getting benefits out of DVC instead of spending time defining the ML pipelines.

> DVC tracks all the dependencies, which helps you iterate on ML models faster without thinking what was affected by your last change.
>> In order to track all the dependencies, DVC finds and reads ALL the DVC-files in a repository and builds a dependency graph (DAG) based on these files.

> This is one of the differences between DVC reproducibility and traditional Makefile-like build automation tools (Make, Maven, Ant, Rakefile etc). It was designed in such a way to localize specification of DAG nodes.
If you run repro on any created DVC-file from our repository, nothing happens because nothing was changed in the defined pipeline.

(c) dvc.org https://dvc.org/doc/tutorial/reproducibility

In [118]:
# Nothing to reproduce
!dvc repro

Stage 'data_load' didn't change, skipping                                       
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


## Experiment 1: Add features



### Create new experiment branch

Before editing the code/featurization.py file, please create and checkout a new branch __ratio_features__

In [119]:
# create new branch

!git checkout -b exp1-ratio-features
!git branch

Switched to a new branch 'exp1-ratio-features'
  dev
  dvc-tutorial
* exp1-ratio-features
  master


### Update featurization.py

in file __featurization.py__  in function`get_features()` after line 

```python
    features = dataset.copy()
```

add lines:

```python
    features['sepal_length_to_sepal_width'] = features['sepal_length'] / features['sepal_width']
    features['petal_length_to_petal_width'] = features['petal_length'] / features['petal_width']
```

### Reproduce pipeline 

In [120]:
!dvc repro

Stage 'data_load' didn't change, skipping                                       
Running stage 'feature_extraction' with command:
	python src/featurization.py --config=params.yaml
Updating lock file 'dvc.lock'                                                   

Restored stage 'split_dataset' from run-cache
Skipping run, checking out outputs
Updating lock file 'dvc.lock'

Restored stage 'train' from run-cache
Skipping run, checking out outputs
Updating lock file 'dvc.lock'

Restored stage 'evaluate' from run-cache
Skipping run, checking out outputs
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock


In [121]:
# Check features used in this pipeline

import pandas as pd

features = pd.read_csv('data/iris_featurized.csv')
features.head()

,sepal_length,sepal_width,petal_length,petal_width,target,sepal_length_to_sepal_width,petal_length_to_petal_width
0,5.1,3.5,1.4,0.2,0,1.457143,7.0
1,4.9,3.0,1.4,0.2,0,1.633333,7.0
2,4.7,3.2,1.3,0.2,0,1.468750,6.5
3,4.6,3.1,1.5,0.2,0,1.483871,7.5
4,5.0,3.6,1.4,0.2,0,1.388889,7.0


In [122]:
!git status

On branch exp1-ratio-features
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   dvc.lock
	modified:   src/featurization.py

no changes added to commit (use "git add" and/or "git commit -a")


In [124]:
# Get difference with metric from previous pipeline
!dvc metrics diff --all

Path               Metric    Value    Change                                    
data/metrics.json  f1_score  0.15385  0.0


In [125]:
!git add .
!git commit -m "Experiment with new features"
!git tag -a "exp1_ratio_features" -m "Experiment with new features"

[exp1-ratio-features 1fc8ec3] Experiment with new features
 3 files changed, 872 insertions(+), 510 deletions(-)
fatal: tag 'exp1_ratio_features' already exists


## Experiment 2: Tune Logistic Regression

### Create a new experiment branch

In [127]:
# create new branch for experiment

!git checkout -b exp2-tuning-logreg
!git branch

Switched to a new branch 'exp2-tuning-logreg'
  dev
  dvc-tutorial
  exp1-ratio-features
* exp2-tuning-logreg
  master


In [129]:
# Nothing to reproduce since code was checked out by `git checkout`
# and data files were checked out by `dvc checkout`
!dvc repro

Stage 'data_load' didn't change, skipping                                       
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


### Tuning parameters

in file __train.py__ :

replace LogisticRegression params with:

```python
    clf = LogisticRegression(C=0.1, solver='newton-cg', multi_class='multinomial', max_iter=100)
```
__Note__: here we changed logistic regresssion hyperparameters: C  to 0.1


https://dvc.org/doc/tutorials/get-started/experiments#tuning-parameters

### Reproduce pipelines

In [130]:
# re-run pipeline 

!dvc repro

Stage 'data_load' didn't change, skipping                                       
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Running stage 'train' with command:
	python src/train.py --config=params.yaml
Updating lock file 'dvc.lock'                                                   

Restored stage 'evaluate' from run-cache
Skipping run, checking out outputs
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock


In [131]:
# Get difference with metric from previous pipeline
!cat data/metrics.json

{"f1_score": 1.0}

In [134]:
!dvc metrics show

	data/metrics.json:                                                             
		f1_score: 1.0


In [135]:
!dvc metrics diff --all

Path               Metric    Value    Change                                    
data/metrics.json  f1_score  1.0      0.84615


### Commit

In [137]:
%%bash

git add .
git commit -m "Tune model. LogisticRegression. C=0.1"
git tag -a "exp2_tuning_logreg" -m "Tune model. LogisticRegression. C=0.1"

On branch exp2-tuning-logreg
nothing to commit, working tree clean


## Experiment 3: Use SVM

### Create a new experiment branch

In [138]:
!git checkout -b exp3-svm
!git branch

Switched to a new branch 'exp3-svm'
  dev
  dvc-tutorial
  exp1-ratio-features
  exp2-tuning-logreg
* exp3-svm
  master


### Update train.py

in file __train.py__ replace line

```python
    clf = LogisticRegression(C=0.1, solver='newton-cg', multi_class='multinomial', max_iter=100)
```

with line

```python
    clf = SVC(C=0.01, kernel='linear', gamma='scale', degree=5)
```


### Reproduce pipeline 

In [139]:
!dvc repro

Stage 'data_load' didn't change, skipping                                       
Stage 'feature_extraction' didn't change, skipping
Stage 'split_dataset' didn't change, skipping
Running stage 'train' with command:
	python src/train.py --config=params.yaml
Updating lock file 'dvc.lock'                                                   

Restored stage 'evaluate' from run-cache
Skipping run, checking out outputs
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock


In [140]:
!dvc metrics show

	data/metrics.json:                                                             
		f1_score: 1.0


In [141]:
!git status

On branch exp3-svm
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   dvc.lock
	modified:   src/train.py

no changes added to commit (use "git add" and/or "git commit -a")


In [142]:
!git add .
!git commit -m "Experiment 3 with SVM estimator"
!git tag -a "exp3_svm" -m "Experiment 3 with SVM estimator"

[exp3-svm 1474ec0] Experiment 3 with SVM estimator
 2 files changed, 5 insertions(+), 4 deletions(-)


## Merge best experiment `dvc-tutorial ` branch

In [153]:
%%bash

git checkout dvc-tutorial 
git merge exp3_svm

Auto-merging src/train.py
CONFLICT (content): Merge conflict in src/train.py
Auto-merging src/featurization.py
CONFLICT (add/add): Merge conflict in dvc.lock
Auto-merging dvc.lock
Auto-merging dvc-3-automate-experiments.ipynb
CONFLICT (content): Merge conflict in dvc-3-automate-experiments.ipynb
Automatic merge failed; fix conflicts and then commit the result.


Previous HEAD position was 1474ec0 Experiment 3 with SVM estimator
Switched to branch 'dvc-tutorial'


CalledProcessError: Command 'b'\ngit checkout dvc-tutorial \ngit merge exp3_svm\n'' returned non-zero exit status 1.

# Compare experiment

## Compare params 

In [147]:
# Get params diffs 

!dvc params diff

In [148]:
# Compare parameters with a specific commit, a tag or any revision

!dvc params diff --all

Path         Param                         Old                       New        
params.yaml  data_load.classes_names_path  data/classes.json         data/classes.json
params.yaml  data_load.raw_data_path       data/iris.csv             data/iris.csv
params.yaml  data_split.test_path          data/test.csv             data/test.csv
params.yaml  data_split.test_size          0.2                       0.2
params.yaml  data_split.train_path         data/train.csv            data/train.csv
params.yaml  evaluate.confusion_matrix     data/cm.csv               data/cm.csv
params.yaml  evaluate.metrics_file         data/metrics.json         data/metrics.json
params.yaml  featurize.features_path       data/iris_featurized.csv  data/iris_featurized.csv
params.yaml  featurize.target_column       target                    target
params.yaml  train.model_path              data/model.joblib         data/model.joblib


In [55]:
!dvc params diff --show-json --all

{"params.yaml": {"evaluate.metrics_file": {"old": "data/metrics.json", "new": "data/metrics.json"}, "featurize.features_path": {"old": "data/iris_featurized.csv", "new": "data/iris_featurized.csv"}, "data_load.classes_names_path": {"old": "data/classes.json", "new": "data/classes.json"}, "data_split.test_path": {"old": "data/test.csv", "new": "data/test.csv"}, "train.model_path": {"old": "data/model.joblib", "new": "data/model.joblib"}, "featurize.target_column": {"old": "target", "new": "target"}, "data_load.raw_data_path": {"old": "data/iris.csv", "new": "data/iris.csv"}, "evaluate.confusion_matrix": {"old": "data/cm.csv", "new": "data/cm.csv"}, "data_split.test_size": {"old": 0.2, "new": 0.2, "diff": 0.0}, "data_split.train_path": {"old": "data/train.csv", "new": "data/train.csv"}}}


In [56]:
!dvc params diff --show-md --all

| Path        | Param                        | Old                      | New                      |
|-------------|------------------------------|--------------------------|--------------------------|
| params.yaml | data_load.classes_names_path | data/classes.json        | data/classes.json        |
| params.yaml | data_load.raw_data_path      | data/iris.csv            | data/iris.csv            |
| params.yaml | data_split.test_path         | data/test.csv            | data/test.csv            |
| params.yaml | data_split.test_size         | 0.2                      | 0.2                      |
| params.yaml | data_split.train_path        | data/train.csv           | data/train.csv           |
| params.yaml | evaluate.confusion_matrix    | data/cm.csv              | data/cm.csv              |
| params.yaml | evaluate.metrics_file        | data/metrics.json        | data/metrics.json        |
| params.yaml | featurize.features_path      | data/iris_featurized.csv | data/iris_featuri

In [28]:
# To see the difference between two specific commits, both need to be specified:

!git log

commit 736c92a6eeda6261f528d7a2e2d4db4cb306fa03 (HEAD -> exp2-svm, tag: exp2_svm)
Author: Mikhail <mnrozhkov@gmail.com>
Date:   Fri Jul 3 21:49:25 2020 +0300

    Experiment 2 with SVM estimator

commit 24f75fdcc9bede20cbecf88697b5d3f8ed56f58c (tag: exp1_ratio_features, exp1-ratio-features)
Author: Mikhail <mnrozhkov@gmail.com>
Date:   Fri Jul 3 21:48:42 2020 +0300

    Experiment with new features

commit 34a0bc667f86c3b5e388bef672eb598b8a6a7788 (dvc-tutorial)
Author: Mikhail <mnrozhkov@gmail.com>
Date:   Wed Jul 1 10:35:03 2020 +0300

    Add stage evaluate

commit 4c45a4ff702106d78bbaf8d356e0e95ca268e05b
Author: Mikhail <mnrozhkov@gmail.com>
Date:   Wed Jul 1 10:34:09 2020 +0300

    Add stage train

commit f41781d2c4855762c4405636491bc014cc00bd20
Author: Mikhail <mnrozhkov@gmail.com>
Date:   Wed Jul 1 10:34:00 2020 +0300

    Add stage split_dataset

commit dbfc854a931baf57ad116f811c2cea39d4fb69a9
Author: Mikhail <mnrozhkov@gmail.com>
Date:   Wed Jul 1 10:33:51 2020 +0300

    Add 

In [30]:

!dvc params diff 24f75fdcc9bede20cbecf88697b5d3f8ed56f58c HEAD^

## Show metrics

In [149]:
# this pipeline metrics 

!dvc metrics show

	data/metrics.json:                                                             
		f1_score: 1.0


In [150]:
# show all commited pipelines metrics (all branch and tags)

!dvc metrics show -a -T

workspace:                                                                      
	data/metrics.json:
		f1_score: 1.0
dev:
	data/metrics.json:
		f1_score: 1.0
dvc-tutorial:
	data/metrics.json:
		f1_score: 0.9305555555555555
exp1-ratio-features:
	data/metrics.json:
		f1_score: 0.15384615384615383
exp2-tuning-logreg:
	data/metrics.json:
		f1_score: 1.0
exp3-svm:
	data/metrics.json:
		f1_score: 1.0
exp2_tuning_logreg:
	data/metrics.json:
		f1_score: 1.0
exp3_svm:
	data/metrics.json:
		f1_score: 1.0


## Compare metrics (get differences)


In [151]:
!dvc metrics diff

In [152]:
# --all - list all metrics, even those without changes

!dvc metrics diff --all

Path               Metric    Value    Change                                    
data/metrics.json  f1_score  1.0      0.0


* чтобы сравнить текущую метрики из текущего коммита и из другого, нужно указать другой (old) коммит:

In [62]:
# Compare old and new branches


!dvc metrics diff exp1-ratio-features exp2-svm

Path               Metric    Value    Change                                    
data/metrics.json  f1_score  1.0      0.84615


In [61]:
# Equivalent to `!dvc metrics diff exp1-ratio-features dvc-tutorial`, because dvc-tutorial - current branch

!dvc metrics diff exp1-ratio-features

Path               Metric    Value    Change                                    
data/metrics.json  f1_score  0.93056  0.77671


In [157]:
!git checkout dev -f

Switched to branch 'dev'
Your branch is ahead of 'origin/dev' by 7 commits.
  (use "git push" to publish your local commits)


* чтобы выводить не только новую, но и старую метрики, нужно добавить опцию --old

In [154]:
# Use --old to show both old and new metrics vlues 

!dvc metrics diff --old exp1-ratio-features exp2-svm

ERROR: failed to show metrics diff - unable to read: 'dvc.lock', YAML file structure is corrupted: while scanning a simple key
  in "<unicode string>", line 22, column 1
could not find expected ':'
  in "<unicode string>", line 23, column 8

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!


In [158]:
!dvc metrics diff --show-md

| Path   | Metric   | Value   | Change   |                                      
|--------|----------|---------|----------|


## Build Plots


In [165]:
from IPython.display import IFrame

### Show

In [176]:
!dvc plots show  --template confusion "data/cm.csv" -x actual -y predicted -o data/plots-show.html

file:///Users/mnrozhkov/dev/dvc/course/dvc-3-automate-experiments/data/plots-show.html


In [177]:
IFrame(src='data/plots-show.html', width=500, height=500)

### Diff

In [192]:
# Build metircs plots for all 3 experiments
!dvc plots diff -t confusion -o data/plots-diff.html exp1-ratio-features exp3-svm -x predicted

file:///Users/mnrozhkov/dev/dvc/course/dvc-3-automate-experiments/data/plots-diff.html


In [194]:
IFrame(src='data/plots-diff.html', width=1000, height=400)